In [2]:
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from datetime import datetime, timedelta

In [13]:
def picnic_weather_full(date1, date2, *places):
    
    weather_dict = []
    
    
    #create a list of dates:
    
    start_date_dt = datetime.strptime(date1, "%Y-%m-%d")
    end_date_dt = datetime.strptime(date2, "%Y-%m-%d")


    delta = end_date_dt - start_date_dt  # returns timedelta

    
    day_list = []
    for i in range(delta.days + 1):
        day = (start_date_dt + timedelta(days=i)).date()
        day = day.strftime("%Y-%m-%d")
    
        day_list.append(day)
    
    for day in day_list:
        
        for loc in places:
            
            #create a script to find latitude and longitude:
            geolocator = Nominatim(user_agent="shree_app")
    
    
            location = geolocator.geocode(loc)
            latitude = location.latitude
            longitude = location.longitude


            #make url
            
            url = f"https://api.brightsky.dev/weather?lat={latitude}&lon={longitude}&date={day}"
            #print(url)
            r = requests.get(url).json()
            weather = r['weather']
            n = len(weather)
            avg_windspeed = np.mean([wind['wind_speed'] for wind in weather if wind['wind_speed'] != None])
            avg_temperature = np.mean([temp['temperature'] for temp in weather if temp['temperature'] != None])
            avg_sunshine = np.mean([sun['sunshine'] for sun in weather if sun['sunshine'] != None])
            avg_precipitation = np.mean([prec['precipitation'] for prec in weather if prec['precipitation'] != None])
            weather_dict.append({'date': day,
                                 'location': loc,
                                        'avg_windspeed': avg_windspeed,
                                        'avg_temperature' : avg_temperature,
                                        'avg_sunshine' : avg_sunshine,
                                        'avg_precipitation' : avg_precipitation})
            
            df = pd.DataFrame.from_dict(weather_dict)
            df_fill = df.fillna(0)
            
            df_new = df_fill[(df_fill['avg_windspeed']<= 30.0) &
            (df_fill['avg_temperature']<= 30.0) &
            (df_fill['avg_temperature']>= 15.0) &
            (df_fill['avg_sunshine']>= 20.0) &
            (df_fill['avg_precipitation']<= 0.05)]
            
            df_result = df_new[['date','location']]
            
            
        
    return df_result

In [14]:
picnic_weather_full('2022-05-14', '2022-05-20', 'treptower park, berlin', 'mitte, berlin', 'steglitz, berlin')

,date,location
5,2022-05-15,"steglitz, berlin"
6,2022-05-16,"treptower park, berlin"
7,2022-05-16,"mitte, berlin"
12,2022-05-18,"treptower park, berlin"
13,2022-05-18,"mitte, berlin"
14,2022-05-18,"steglitz, berlin"
15,2022-05-19,"treptower park, berlin"
16,2022-05-19,"mitte, berlin"
17,2022-05-19,"steglitz, berlin"
18,2022-05-20,"treptower park, berlin"
